### Case summary

In [ ]:
import os, sys
import numpy as np
from shutil import rmtree, copy
from matplotlib import pyplot as plt
from matplotlib import gridspec, cm

# directory to the aspect Lab
ASPECT_LAB_DIR = os.environ['ASPECT_LAB_DIR']
TwoDSubduction_DIR = os.environ['TwoDSubduction_DIR']
ThDSubduction_DIR = os.environ['ThDSubduction_DIR']
RESULT_DIR = os.path.join(ASPECT_LAB_DIR, 'results')

sys.path.append(os.path.join(ASPECT_LAB_DIR))
from shilofue.ThDSubduction0.Group import CASE_SUMMARY
from shilofue.TwoDSubduction0.Group import CASE_SUMMARY as CASE_SUMMARY_2D
import shilofue.ThDSubduction0.VtkPp as ThDVtkPp
from shilofue.Group import ReadBasicInfoGroup
from shilofue.ThDSubduction0.PlotVisit import VISIT_OPTIONS
from shilofue.TwoDSubduction0.PlotVisit import VISIT_OPTIONS as VISIT_OPTIONS_2D

In [ ]:
group_dir = os.path.join(ThDSubduction_DIR, "chunk_test")
assert(os.path.isdir(group_dir))
group_dir_2d = os.path.join(TwoDSubduction_DIR, "EBA_CDPT_3dconsistent_9")
assert(os.path.isdir(group_dir_2d))

### 3d case summary

In [ ]:
all_actions=['geometry', 't660', 't800', 't1000', 'dip660']

# for the project, no update, hand correct
update_from_group = False; o_path = os.path.join(ThDSubduction_DIR, 'case_summary.txt'); o_path_csv = os.path.join(ThDSubduction_DIR, 'case_summary.csv'); o_path_tex = os.path.join(ThDSubduction_DIR, 'case_summary.tex')
# for one group, update from group
update_from_group = True; o_path = os.path.join(group_dir, 'case_summary.txt'); o_path_csv = os.path.join(group_dir, 'case_summary.csv'); o_path_tex = os.path.join(group_dir, 'case_summary.tex')

img_dir = os.path.join(group_dir, 'img')
if not os.path.isdir(img_dir):
  os.mkdir(img_dir)

Case_Summary = CASE_SUMMARY(VISIT_OPTIONS=VISIT_OPTIONS)

# import old result if it exists 
if os.path.isfile(o_path_csv):
  Case_Summary.import_file(o_path_csv)
  Case_Summary.Update(actions=all_actions)

# import the new directory
if update_from_group:
  Case_Summary.import_directory(group_dir, actions=all_actions)

# output file output 
Case_Summary.write_file(o_path)
Case_Summary.write_file(o_path_csv)
Case_Summary.write_file(o_path_tex)

### 2d case summary

In [ ]:
# for the project, no update, hand correct
update_from_group = False; o_path = os.path.join(ThDSubduction_DIR, 'case_summary_2d.txt'); o_path_csv = os.path.join(ThDSubduction_DIR, 'case_summary_2d.csv'); o_path_tex = os.path.join(ThDSubduction_DIR, 'case_summary_2d.tex')
# for one group, update from group
update_from_group = True; o_path = os.path.join(group_dir_2d, 'case_summary_2d.txt'); o_path_csv = os.path.join(group_dir_2d, 'case_summary_2d.csv'); o_path_tex = os.path.join(group_dir_2d, 'case_summary_2d.tex')

all_actions=["t660", "t800", "t1000", "dip660"]

img_dir = os.path.join(group_dir_2d, 'img')
if not os.path.isdir(img_dir):
  os.mkdir(img_dir)

Case_Summary_2d = CASE_SUMMARY_2D(VISIT_OPTIONS=VISIT_OPTIONS_2D)

# import old result if it exists 
if os.path.isfile(o_path_csv):
  Case_Summary_2d.import_file(o_path_csv)
  Case_Summary_2d.Update(actions=all_actions)

# import the new directory
if update_from_group:
  Case_Summary_2d.import_directory(group_dir_2d, actions=all_actions)

# output file output 
Case_Summary_2d.write_file(o_path)
Case_Summary_2d.write_file(o_path_csv)
Case_Summary_2d.write_file(o_path_tex)